In [2]:
import numpy as np
import cv2

In [3]:
corner_track_params = dict(maxCorners=10,qualityLevel=0.3,minDistance=7,blockSize=7)

In [4]:
corner_track_params

{'maxCorners': 10, 'qualityLevel': 0.3, 'minDistance': 7, 'blockSize': 7}

In [5]:
lk_params = dict(winSize=(200,200),maxLevel=2,criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))

In [6]:
lk_params

{'winSize': (200, 200), 'maxLevel': 2, 'criteria': (3, 10, 0.03)}

In [7]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame,cv2.COLOR_BGR2GRAY)

#PTS TO TRACK
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask = np.zeros_like(prev_frame)

while True:
    ret, frame = cap.read()
    
    frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params)
    
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new , y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask,(x_new,y_new),(x_prev,y_prev),(0,255,0),3)
        
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
    
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()


In [8]:
import numpy as np
import cv2

In [9]:
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255 #satuation


while True:
    ret, frame2 = cap.read()

    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg,None,0.5,3,15,3,5,1.2,0)
    
    mag , ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True)

    hsv_mask[:,:,0] = ang/2  #hue
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)  #value

    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',bgr)
    
    k = cv2.waitKey(10) & 0xFF
    if k == 27:
        break
    
    prvsImg = nextImg
        
cv2.destroyAllWindows()
cap.release()    

In [6]:
import cv2
import numpy as np
import sys
# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier('DATA/haarcascades/haarcascade_frontalface_default.xml')

# Parameters for ShiTomasi corner detection (within face region)
corner_track_params = dict(
    maxCorners=100,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7
)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(
    winSize=(15, 15),
    maxLevel=2,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03)
)

cap = cv2.VideoCapture(0)
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Detect face
faces = face_cascade.detectMultiScale(prev_gray)

if len(faces) == 0:
    print("No face detected.")
    cap.release()
    cv2.destroyAllWindows()
    exit()
else:
    # Select the first face detected
    (x, y, w, h) = faces[0]
    face_roi = prev_gray[y:y+h, x:x+w]

    # Detect good features (corners) within face
    prevPts = cv2.goodFeaturesToTrack(face_roi, mask=None, **corner_track_params)

    # Offset points to match original frame coordinates
    if prevPts is not None:
        prevPts += np.array([[x, y]])

    mask = np.zeros_like(prev_frame)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)

        # Keep only good points
        good_new = nextPts[status == 1]
        good_prev = prevPts[status == 1]

        for i, (new, prev) in enumerate(zip(good_new, good_prev)):
            x_new, y_new = new.ravel().astype(int)
            x_prev, y_prev = prev.ravel().astype(int)

            mask = cv2.line(mask, (x_new, y_new), (x_prev, y_prev), (0, 255, 0), 2)
            frame = cv2.circle(frame, (x_new, y_new), 4, (0, 0, 255), -1)

        img = cv2.add(frame, mask)
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Draw initial face bounding box
        cv2.imshow('Face Tracking', img)

        k = cv2.waitKey(30) & 0xFF
        if k == 27:
            break

        prev_gray = frame_gray.copy()
        prevPts = good_new.reshape(-1, 1, 2)

    cap.release()
    cv2.destroyAllWindows()

print("AAA")

AAA


In [3]:
print('A')

A
